In [2]:
import numpy as np
import pandas as pd
import requests
import json
import re

In [53]:
url = "http://realtime.mbta.com/developer/api/v2/routes"

querystring = {"api_key":"wX9NwuHnZU2ToO7GmGR9uw","format":"json"}

headers = {
    'cache-control': "no-cache",
    'postman-token': "4c7b8b89-58b4-c8f5-2509-0647e1775340"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
routes=response.json()
#Route Types: Subway, Commuter Rail, Bus, Boat
subwayRouteID=[[y['route_id'] for y in x['route']] for x in routes['mode'] if x['mode_name']=='Subway']
subwayRouteID=[y for x in subwayRouteID for y in x]
crRouteID=[[y['route_id'] for y in x['route']] for x in routes['mode'] if x['mode_name']=='Commuter Rail']
crRouteID=[y for x in crRouteID for y in x]
busRouteID=[[y['route_id'] for y in x['route']] for x in routes['mode'] if x['mode_name']=='Bus']
busRouteID=[y for x in busRouteID for y in x]
boatRouteID=[[y['route_id'] for y in x['route']] for x in routes['mode'] if x['mode_name']=='Boat']
boatRouteID=[y for x in boatRouteID for y in x]

routesDic={'Subway':{subkey:[] for subkey in subwayRouteID},'Bus':{buskey:[] for buskey in busRouteID},'Commuter Rail':{crkey:[] for crkey in crRouteID},
           'Boat':{boatkey:[] for boatkey in boatRouteID}}

for key in routesDic:
    for route in routesDic[key]:
        url = "http://realtime.mbta.com/developer/api/v2/stopsbyroute"
        querystring = {"api_key":"wX9NwuHnZU2ToO7GmGR9uw","route":route,"format":"json"}
        headers = {
            'cache-control': "no-cache",
            'postman-token': "014e80a3-7552-8a15-6796-afa8e81495f3"
            }
        response = requests.request("GET", url, headers=headers, params=querystring)
        stops=response.json()
        #routesDic[key][route]=[re.sub(r'(.*) - .*',r'\1',x['stop_name']).replace(' Busway','').replace(' Lower','').replace(' Upper','') for x in stops['direction'][0]['stop']]
        routesDic[key][route]=[re.sub(r'(.*) - .*',r'\1',x['stop_name']).replace(' Busway','').replace(' Lower','').replace(' Upper','') for x in stops['direction'][0]['stop']]
        if key=='Subway':
            routesDic[key][route]=[re.sub(r'(.*) - .*',r'\1',x).replace(' Orange Line','') for x in routesDic[key][route]]
        else:
            routesDic[key][route]=[re.sub(r'(.*) - .*',r'\1',x).replace(' Station','') for x in routesDic[key][route] if 'South' not in x and 'North' not in x]
routesDic['Subway']['Red-Ashmont']=routesDic['Subway']['Red'][:12]+routesDic['Subway']['Red'][13:18]
routesDic['Subway']['Red-Braintree']=routesDic['Subway']['Red'][:13]+routesDic['Subway']['Red'][18:]
routesDic


{'Boat': {'Boat-F1': ['Long Wharf, Boston',
   'Rowes Wharf, Boston',
   'Logan Airport Ferry Terminal',
   "George's Island",
   'Pemberton Point, Hull',
   "Hewitt's Cove, Hingham"],
  'Boat-F4': ['Long Wharf, Boston', 'Charlestown Navy Yard']},
 'Bus': {'1': ['Dudley',
   'Washington St opp Ruggles St',
   'Washington St @ Melnea Cass Blvd',
   'Melnea Cass Blvd @ Harrison Ave',
   'Albany St opp Randall St',
   'Massachusetts Ave @ Albany St',
   'Massachusetts Ave @ Harrison Ave',
   'Massachusetts Ave @ Washington St',
   'Massachusetts Ave @ Tremont St',
   'Massachusetts Ave @ Columbus Ave',
   'Massachusetts Ave @ Massachusetts Ave',
   'Massachusetts Ave @ St Botolph St',
   'Massachusetts Ave @ Clearway St',
   'Massachusetts Ave @ Newbury St',
   'Massachusetts Ave @ Beacon St',
   'Massachusetts Ave @ Memorial Dr',
   '77 Massachusetts Ave',
   'Massachusetts Ave @ Albany St',
   'Massachusetts Ave @ Sidney St',
   'Massachusetts Ave @ Prospect St',
   'Massachusetts Ave @

In [42]:
babyRouteDic={'Subway':{'green':['A','B','P','G'],'red':['P','D']}}
print(babyRouteDic)
stops=[babyRouteDic['Subway'][key] for key in babyRouteDic['Subway']] 
stops=np.unique([y for x in stops for y in x])
adjacencyList={key:[] for key in stops}
for key in babyRouteDic:
    for route in babyRouteDic[key]:
        for i in range(len(babyRouteDic[key][route])):
            if i==0:
                adjacencyList[babyRouteDic[key][route][i]].append(babyRouteDic[key][route][i+1])
            elif i>0 and i<(len(babyRouteDic[key][route])-2):
                adjacencyList[babyRouteDic[key][route][i]].append(babyRouteDic[key][route][i-1])
                adjacencyList[babyRouteDic[key][route][i]].append(babyRouteDic[key][route][i+1])
            else:
                adjacencyList[babyRouteDic[key][route][i]].append(babyRouteDic[key][route][i-1])
adjacencyList

{'Subway': {'red': ['P', 'D'], 'green': ['A', 'B', 'P', 'G']}}


{'A': ['B'], 'B': ['A', 'P'], 'D': ['P'], 'G': ['P'], 'P': ['D', 'B']}

In [52]:
subwayStops=[routesDic['Subway'][key] for key in routesDic['Subway']]
subwayStops=np.unique([y for x in subwayStops for y in x])
subwayAdjacencyList={key:[] for key in subwayStops}
for route in routesDic['Subway']:
    for i in range(len(routesDic['Subway'][route])):
        if i==0:
            if routesDic['Subway'][route][i+1] not in subwayAdjacencyList[routesDic['Subway'][route][i]]:
                subwayAdjacencyList[routesDic['Subway'][route][i]].append(routesDic['Subway'][route][i+1])
        elif i>0 and i<(len(routesDic['Subway'][route])-2):
            if routesDic['Subway'][route][i-1] not in subwayAdjacencyList[routesDic['Subway'][route][i]]:
                subwayAdjacencyList[routesDic['Subway'][route][i]].append(routesDic['Subway'][route][i-1])
            if routesDic['Subway'][route][i+1] not in subwayAdjacencyList[routesDic['Subway'][route][i]]:
                subwayAdjacencyList[routesDic['Subway'][route][i]].append(routesDic['Subway'][route][i+1])
        else:
            if routesDic['Subway'][route][i-1] not in subwayAdjacencyList[routesDic['Subway'][route][i]]:
                subwayAdjacencyList[routesDic['Subway'][route][i]].append(routesDic['Subway'][route][i-1])
subwayAdjacencyList

{'Airport': ['Wood Island', 'Maverick'],
 'Alewife': ['Davis'],
 'Allston Street': ['Griggs Street', 'Warren Street'],
 'Andrew': ['Broadway', 'JFK/UMASS Braintree', 'JFK/UMASS Ashmont'],
 'Aquarium': ['Maverick', 'State Street'],
 'Arlington': ['Boylston', 'Copley'],
 'Ashmont': ['Shawmut', 'North Quincy', 'Cedar Grove'],
 'Assembly': ['Wellington', 'Sullivan Square'],
 'Babcock Street': ['Pleasant Street', 'Packards Corner'],
 'Back Bay': ['Tufts Medical Center', 'Massachusetts Avenue'],
 'Back of the Hill': ['Riverway'],
 'Beachmont': ['Revere Beach', 'Suffolk Downs'],
 'Beaconsfield': ['Brookline Hills', 'Reservoir'],
 'Blandford Street': ['Kenmore', 'Boston Univ East'],
 'Boston College': ['South Street'],
 'Boston Univ Central': ['Boston Univ East', 'Boston Univ West'],
 'Boston Univ East': ['Blandford Street', 'Boston Univ Central'],
 'Boston Univ West': ['Boston Univ Central', 'Saint Paul Street'],
 'Bowdoin': ['Government Center'],
 'Boylston': ['Park Street', 'Arlington'],
 '

In [55]:
busStops=[routesDic['Bus'][key] for key in routesDic['Bus']]
busStops=np.unique([y for x in busStops for y in x])
busAdjacencyList={key:[] for key in busStops}
for route in routesDic['Bus']:
    for i in range(len(routesDic['Bus'][route])):
        if i==0:
            if routesDic['Bus'][route][i+1] not in busAdjacencyList[routesDic['Bus'][route][i]]:
                busAdjacencyList[routesDic['Bus'][route][i]].append(routesDic['Bus'][route][i+1])
        elif i>0 and i<(len(routesDic['Bus'][route])-2):
            if routesDic['Bus'][route][i-1] not in busAdjacencyList[routesDic['Bus'][route][i]]:
                busAdjacencyList[routesDic['Bus'][route][i]].append(routesDic['Bus'][route][i-1])
            if routesDic['Bus'][route][i+1] not in busAdjacencyList[routesDic['Bus'][route][i]]:
                busAdjacencyList[routesDic['Bus'][route][i]].append(routesDic['Bus'][route][i+1])
        else:
            if routesDic['Bus'][route][i-1] not in busAdjacencyList[routesDic['Bus'][route][i]]:
                busAdjacencyList[routesDic['Bus'][route][i]].append(routesDic['Bus'][route][i-1])
busAdjacencyList

{'5 Centennial Dr': ['Fourth St @ First Ave', 'Centennial Dr @ Technology Dr'],
 'Readville St @ Como Rd': ['River St @ Solaris Rd',
  'River St @ Milton St',
  'Readville St @ Norton St',
  'Readville St @ W Milton St'],
 'Dorchester Ave @ Thornley Rd': ['Dorchester Ave @ Pearl St',
  'Dorchester Ave @ Melvinside Ter'],
 'Burns St @ Western Ave': ['Summer St @ River St', 'Western Ave @ Minot St'],
 'Salem St @ Cutler Hwy': ['Lynn St @ Spring St',
  'Salem St @ Waitt Pk',
  'Salem St @ Waitt Ct',
  'Lynn St @ Lawrence St',
  'Salem St @ Lynn St'],
 'Walnut St @ Commonwealth Ave': ['Walnut St @ Prospect Ave',
  'Walnut St @ Homer St'],
 'Lynn St @ Spring Pond Rd': ['Broadway @ Bay State Rd',
  'Lynn St opp Fairview Ave'],
 'Geneva Ave @ Holiday St': ['Adams St @ Centre St',
  'Geneva Ave @ Westville St',
  'Bowdoin St @ Geneva Ave'],
 'Beaver St opp Forest St': ['Forest St @ Beaver St', 'Beaver St @ Field Rd'],
 '851 Pleasant St': ['1197 Washington St'],
 'Willow St @ Lyme St': ['Bowdoi

In [62]:
crStops=[routesDic['Commuter Rail'][key] for key in routesDic['Commuter Rail']]
crStops=np.unique([y for x in crStops for y in x])
crAdjacencyList={key:[] for key in crStops}
for route in routesDic['Commuter Rail']:
    for i in range(len(routesDic['Commuter Rail'][route])):
        if i==0:
            if routesDic['Commuter Rail'][route][i+1] not in crAdjacencyList[routesDic['Commuter Rail'][route][i]]:
                crAdjacencyList[routesDic['Commuter Rail'][route][i]].append(routesDic['Commuter Rail'][route][i+1])
        elif i>0 and i<(len(routesDic['Commuter Rail'][route])-2):
            if routesDic['Commuter Rail'][route][i-1] not in crAdjacencyList[routesDic['Commuter Rail'][route][i]]:
                crAdjacencyList[routesDic['Commuter Rail'][route][i]].append(routesDic['Commuter Rail'][route][i-1])
            if routesDic['Commuter Rail'][route][i+1] not in crAdjacencyList[routesDic['Commuter Rail'][route][i]]:
                crAdjacencyList[routesDic['Commuter Rail'][route][i]].append(routesDic['Commuter Rail'][route][i+1])
        else:
            if routesDic['Commuter Rail'][route][i-1] not in crAdjacencyList[routesDic['Commuter Rail'][route][i]]:
                crAdjacencyList[routesDic['Commuter Rail'][route][i]].append(routesDic['Commuter Rail'][route][i-1])
crAdjacencyList

{'Abington': ['Braintree', 'Whitman'],
 'Anderson/Woburn': ['Melrose Highlands',
  'Greenwood',
  'Mishawum',
  'Wilmington'],
 'Andover': ['Ballardvale', 'Lawrence'],
 'Ashland': ['Framingham', 'Westborough'],
 'Attleboro': ['Stoughton', 'Providence'],
 'Auburndale': ['West Newton', 'Wellesley Farms'],
 'Ayer': ['Littleton/Rte 495', 'Shirley'],
 'Back Bay': ['Yawkey', 'Ruggles'],
 'Ballardvale': ['Reading', 'Andover'],
 'Bellevue': ['Roslindale Village', 'Highland'],
 'Belmont': ['Porter Square', 'Waverley'],
 'Beverly': ['Salem', 'Montserrat'],
 'Beverly Farms': ['Ipswich', 'Manchester'],
 'Boston Landing': ['Yawkey', 'Newtonville'],
 'Bradford': ['Lawrence'],
 'Braintree': ['Quincy Center', 'Abington', 'Holbrook/Randolph'],
 'Brandeis/Roberts': ['Waltham', 'Kendal Green'],
 'Bridgewater': ['Campello'],
 'Brockton': ['Montello', 'Campello'],
 'Campello': ['Brockton', 'Bridgewater'],
 'Canton Center': ['Canton Junction', 'Sharon'],
 'Canton Junction': ['Route 128', 'Canton Center'],
 

In [65]:
fStops=[routesDic['Boat'][key] for key in routesDic['Boat']]
fStops=np.unique([y for x in fStops for y in x])
fAdjacencyList={key:[] for key in fStops}
for route in routesDic['Boat']:
    for i in range(len(routesDic['Boat'][route])):
        if i==0:
            if routesDic['Boat'][route][i+1] not in fAdjacencyList[routesDic['Boat'][route][i]]:
                fAdjacencyList[routesDic['Boat'][route][i]].append(routesDic['Boat'][route][i+1])
        elif i>0 and i<(len(routesDic['Boat'][route])-2):
            if routesDic['Boat'][route][i-1] not in fAdjacencyList[routesDic['Boat'][route][i]]:
                fAdjacencyList[routesDic['Boat'][route][i]].append(routesDic['Boat'][route][i-1])
            if routesDic['Boat'][route][i+1] not in fAdjacencyList[routesDic['Boat'][route][i]]:
                fAdjacencyList[routesDic['Boat'][route][i]].append(routesDic['Boat'][route][i+1])
        else:
            if routesDic['Boat'][route][i-1] not in fAdjacencyList[routesDic['Boat'][route][i]]:
                fAdjacencyList[routesDic['Boat'][route][i]].append(routesDic['Boat'][route][i-1])
fAdjacencyList

{'Charlestown Navy Yard': ['Long Wharf, Boston'],
 "George's Island": ['Logan Airport Ferry Terminal', 'Pemberton Point, Hull'],
 "Hewitt's Cove, Hingham": ['Pemberton Point, Hull'],
 'Logan Airport Ferry Terminal': ['Rowes Wharf, Boston', "George's Island"],
 'Long Wharf, Boston': ['Charlestown Navy Yard', 'Rowes Wharf, Boston'],
 'Pemberton Point, Hull': ["George's Island"],
 'Rowes Wharf, Boston': ['Long Wharf, Boston', 'Logan Airport Ferry Terminal']}